# load the dataset and do some basic preprocessing

In [6]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs
!pip install -U datasets

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", bu

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
from datasets import load_dataset


raw_datasets = load_dataset("code_search_net", "python")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

code_search_net.py: 0.00B [00:00, ?B/s]

In [6]:
# transform the dataset into an iterator list of lists for effecient training
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]


training_corpus = get_training_corpus()

# Training a new Tokenizer


In [7]:
from transformers import AutoTokenizer

# load the tokenizer architecture
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
# run the training
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [17]:
# test
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = tokenizer.tokenize(example)
tokens

#compare
print("new tokenizer(trained): ", len(tokens))
print("old tokenizer(not trained): ", len(old_tokenizer.tokenize(example)))

new tokenizer(trained):  27
old tokenizer(not trained):  36


In [19]:
#save the tokenizer
tokenizer.save_pretrained("code-search-net-tokenizer")
#push to hub
tokenizer.push_to_hub("code-search-net-tokenizer")

CommitInfo(commit_url='https://huggingface.co/pranavsaranaway/code-search-net-tokenizer/commit/e9f6e4bb415c89d1e60ea865579f4cec728e8b89', commit_message='Upload tokenizer', commit_description='', oid='e9f6e4bb415c89d1e60ea865579f4cec728e8b89', pr_url=None, repo_url=RepoUrl('https://huggingface.co/pranavsaranaway/code-search-net-tokenizer', endpoint='https://huggingface.co', repo_type='model', repo_id='pranavsaranaway/code-search-net-tokenizer'), pr_revision=None, pr_num=None)